In [ ]:
import os
import glob
import cv2
import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
import tensorflow as tf

#'~/Documents/Machine Learning/Person homework/training.csv
FTRAIN='training.csv'
FTEST='test.csv'

#将所有的图片的尺寸重现设置为32*32
w=96
h=96
c=1

"""Loads data from FTEST if *test* is True,otherwise from FTRAIN
Padd a list of *cols* if you are only interested in a subset of the target columns"""
def load(test=False,cols=None):
    
    fname=FTEST if test else FTRAIN
    #load pandas dataframe
    df=read_csv(os.path.expanduser(fname))
    
    #The Image column has piazxel values separated by space:convert
    #the values to numpy arrays:
    df['Image']=df['Image'].apply(lambda im: np.fromstring(im,sep=' '))
    
    #get a subset of columns
    if cols:
        df=df[list(cols)+['Image']]
    
    #print the number of values for each column
    #print(df.count())
    #drop all rows that have missing values in them
    df=df.dropna()
    
    #scale pixel values to [0,1]
    train_data=np.vstack(df['Image'].values)/255
    train_data=train_data.astype(np.float32)
    
    #only FTRAIN has any target columns
    if not test:
        train_label=df[df.columns[:-1]].values
        #scale target coordiates to [-1,1]
        train_label=(train_label-48)/48
        #shuffle train data
        train_data,train_label=shuffle(train_data,train_label,random_state=42)
        train_label=train_label.astype(np.float32)
    else:
        train_label=None
        
    return train_data,train_label

def load2d(test=False,cols=None):
    X,y=load(test=test)
    #X=X.reshape(-1,1,96,96)
    
    X=X.reshape(-1,w,h,c)
    return X,y

def load_show(test=False,cols=None):
    
    fname=FTEST if test else FTRAIN
    #load pandas dataframe
    df=read_csv(os.path.expanduser(fname))
    
    #The Image column has piazxel values separated by space:convert
    #the values to numpy arrays:
    df['Image']=df['Image'].apply(lambda im: np.fromstring(im,sep=' '))
    
    #get a subset of columns
    if cols:
        df=df[list(cols)+['Image']]
    
    #print the number of values for each column
    #print(df.count())
    #drop all rows that have missing values in them
    df=df.dropna()
    
    #scale pixel values to [0,1]
    train_data=np.vstack(df['Image'].values)
    train_data=train_data.astype(np.float32)
    
    #only FTRAIN has any target columns
    if not test:
        train_label=df[df.columns[:-1]].values
        #scale target coordiates to [-1,1]
        train_label=(train_label-48)/48
        #shuffle train data
        train_data,train_label=shuffle(train_data,train_label,random_state=42)
        train_label=train_label.astype(np.float32)
    else:
        train_label=None
        
    return train_data,train_label

def load2d_show(test=False,cols=None):
    X,y=load_show(test=test)
    #X=X.reshape(-1,1,96,96)
    
    X=X.reshape(-1,w,h,c)
    return X,y

#test load data
train_data,train_label=load2d()
test_data,test_label=load2d(test=True)
print("train_data.shape=={};train_data.min=={:.3f};train_data.max={:.3f}".format(train_data.shape,train_data.min(),train_data.max()))
print("train_label.shape=={};train_label.min=={:.3f};train_label.max={:.3f}".format(train_label.shape,train_label.min(),train_label.max()))
print("test_data.shape=={};".format(test_data.shape))


#打乱训练数据及测试数据
train_image_num=len(train_data)
train_image_index=np.arange(train_image_num)
np.random.shuffle(train_image_index)
train_data=train_data[train_image_index]
train_label=train_label[train_image_index]

test_image_num=len(test_data)
test_image_index=np.arange(test_image_num)
np.random.shuffle(test_image_index)
test_data=test_data[test_image_index]
#test_label=test_label[test_image_index]

keypoint_index={
    'left_eye_center_x':0,
    'left_eye_center_y':1,
    'right_eye_center_x':2,
    'right_eye_center_y':3,
    'left_eye_inner_corner_x':4,
    'left_eye_inner_corner_y':5,
    'left_eye_outer_corner_x':6,
    'left_eye_outer_corner_y':7,
    'right_eye_inner_corner_x':8,
    'right_eye_inner_corner_y':9,
    'right_eye_outer_corner_x':10,
    'right_eye_outer_corner_y':11,
    'left_eyebrow_inner_end_x':12,
    'left_eyebrow_inner_end_y':13,
    'left_eyebrow_outer_end_x':14,
    'left_eyebrow_outer_end_y':15,
    'right_eyebrow_inner_end_x':16,
    'right_eyebrow_inner_end_y':17,
    'right_eyebrow_outer_end_x':18,
    'right_eyebrow_outer_end_y':19,
    'nose_tip_x':20,
    'nose_tip_y':21,
    'mouth_left_corner_x':22,
    'mouth_left_corner_y':23,
    'mouth_right_corner_x':24,
    'mouth_right_corner_y':25,
    'mouth_center_top_lip_x':26,
    'mouth_center_top_lip_y':27,
    'mouth_center_bottom_lip_x':28,
    'mouth_center_bottom_lip_y':29
}

#搭建cnn
#定义形参，在执行过程中被赋值
#[None,30]None代表行不确定，列是30
x=tf.placeholder(tf.float32,[None,w,h,c],name='x')
y_=tf.placeholder(tf.float32,[None,30],name='y_')
keep_prob=tf.placeholder(tf.float32,name='keep_prob')

def inference(input_tensor,train,regularizer):
    
    print("begin_inference1")
    #第一层：卷积层，filter（3*3*6），padding=0，stride=1
    #scale change:96*96*1->94*94*32
    #返回一个对象
    #with tf.variable_scope('layer1-conv1',reuse=True):
    with tf.variable_scope('layer1-conv1'):
        #truncated_normal_initializer从截断的正态分布中输出随机值。(stddev :方差)
        #[3,3,c,32]:[filter_size_w,filter_size_h,old_c_num,new_c_num]
        conv1_weights=tf.get_variable('weight',[3,3,c,32],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases=tf.get_variable('bias',[32],initializer=tf.constant_initializer(0.0))
        #tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
        conv1=tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='VALID')
        #tf.nn.relu(features, name=None)
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
        
    #第二层：池化层，filter（2×2），paddig=0，stride=2
    #scale change:94*94*32->47*47*32
    with tf.name_scope('layer2_pool1'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        
    #第三层：卷积层，filter（2×2×64），padding=0，stride=1
    #scale change:47*47*32->46*46*64
    #返回一个对象
    #with tf.variable_scope('layer3-conv2',reuse=True):
    with tf.variable_scope('layer3-conv2'):
        #truncated_normal_initializer从截断的正态分布中输出随机值。(stddev :方差)
        conv2_weights=tf.get_variable('weight',[2,2,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases=tf.get_variable('bias',[64],initializer=tf.constant_initializer(0.0))
        #tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
        conv2=tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='VALID')
        #tf.nn.relu(features, name=None)
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
        
     #第四层：池化层，filter（2×2），paddig=0，stride=2
    #scale change:46*46*64->23*23*64
    with tf.name_scope('layer4_pool2'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        
     #第五层：卷积层，filter（2×2×128），padding=0，stride=1
    #scale change:23*23*64->22*22*128
    #返回一个对象
    #with tf.variable_scope('layer3-conv2',reuse=True):
    with tf.variable_scope('layer5-conv3'):
        #truncated_normal_initializer从截断的正态分布中输出随机值。(stddev :方差)
        conv3_weights=tf.get_variable('weight',[2,2,64,128],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv3_biases=tf.get_variable('bias',[128],initializer=tf.constant_initializer(0.0))
        #tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None, data_format=None, name=None)
        conv3=tf.nn.conv2d(pool2,conv3_weights,strides=[1,1,1,1],padding='VALID')
        #tf.nn.relu(features, name=None)
        relu3=tf.nn.relu(tf.nn.bias_add(conv3,conv3_biases))
        
     #第六层：池化层，filter（2×2），paddig=0，stride=2
    #scale change:22*22*128->11*11*128
    with tf.name_scope('layer6_pool3'):
        pool3=tf.nn.max_pool(relu3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    #layer6_pool3(11*11*128=15488)->full_connected_layer(一维向量 15488)
    pool_shape=pool3.get_shape().as_list()
    nodes=pool_shape[1]*pool_shape[2]*pool_shape[3]
    reshaped=tf.reshape(pool3,[-1,nodes])
    
    #第七层：全连接层：nodes=11*11*128=15488，15488->500
    #是否引入dropout，此过程中未引入
    #with tf.variable_scope('layer7-fc1',reuse=True):
    with tf.variable_scope('layer7-fc1'):
        fc1_weights=tf.get_variable('weight',[nodes,500],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases=tf.get_variable('bias',[500],initializer=tf.constant_initializer(0.1))
        #tf.matmul()矩阵乘
        fc1=tf.nn.relu(tf.matmul(reshaped,fc1_weights)+fc1_biases)
        if train:
            fc1=tf.nn.dropout(fc1,0.5)
    
    #第八层：全连接层：500->500
    #是否引入dropout，此过程中未引入
    #with tf.variable_scope('layer8-fc2',reuse=True):
    with tf.variable_scope('layer8-fc2'):
        fc2_weights=tf.get_variable('weight',[500,500],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_biases=tf.get_variable('bias',[500],initializer=tf.truncated_normal_initializer(stddev=0.1))
        fc2=tf.nn.relu(tf.matmul(fc1,fc2_weights)+fc2_biases)
        if train:
            fc2=tf.nn.dropout(fc2,0.5)
    
    #第九层：全连接层：500->30
    #get the final classification result
    #with tf.variable_scope('layer7-fc3',reuse=True):
    with tf.variable_scope('layer9-fc3'):
        fc3_weights=tf.get_variable('weight',[500,30],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc3_weights))
        fc3_biases=tf.get_variable('bias',[30],initializer=tf.truncated_normal_initializer(stddev=0.1))
        logit=tf.matmul(fc2,fc3_weights)+fc3_biases
        #??
        rmse=tf.sqrt(tf.reduce_mean(tf.square(y_-logit)))
        
    return logit,rmse

def read_from_file(filename):
    with open(filename) as fd:
        lines = fd.readlines()
        
    return lines

#每次获取batch_size个样本进行训练
def get_train_batch(data,label,batch_size):
    #range范围[0,len（data）-batch_size]，每隔batch_size取一个数据
    for start_index in range(0,len(data)-batch_size+1,batch_size):
        #设置一个batch_size大小的切片
        slice_index=slice(start_index,start_index+batch_size)
        #yield暂停循环，完成后返回循环
        yield data[slice_index],label[slice_index]
        
#每次获取batch_size个样本进行测试
def get_test_batch(data,batch_size):
    #range范围[0,len（data）-batch_size]，每隔batch_size取一个数据
    for start_index in range(0,len(data)-batch_size+1,batch_size):
        #设置一个batch_size大小的切片
        slice_index=slice(start_index,start_index+batch_size)
        #yield暂停循环，完成后返回循环
        yield data[slice_index]
        
        
#在直接执行脚本时调用，import时不用调用
#if __name__=='__main__':
print("loss_begin")
#正则化
regularizer=tf.contrib.layers.l2_regularizer(0.001)
y,loss=inference(x,False,regularizer)
train_op=tf.train.AdamOptimizer(0.001).minimize(loss)

#创建会话
sess=tf.InteractiveSession()
print("begin_session")
#初始化所有的变量
sess.run(tf.global_variables_initializer())train_op
    
    
#train_process
#将所有的样本训练10次，每次训练中以64次为一组
train_num=1000
batch_size=64
validation_size=100
EARLY_STOP_PATIENCE=20
best_validation_loss=1000000.0
current_epoch=0
#确定验证集
valid_data,valid_label=train_data[:validation_size],train_label[:validation_size]
#出去验证集后的训练集
_train_data,_train_label=train_data[validation_size:],train_label[validation_size:]

for i in range(train_num):
    #print("epoch:=={}".format(i))
              
    train_loss,train_acc,batch_num=0,0,0
    for train_data_batch,train_label_batch in get_train_batch(train_data,train_label,batch_size):
        #sess.run(train_op,feed_dict={x:train_data_batch,y_:train_label_batch,keep_prob:0.5})
        train_op.run(feed_dict={x:train_data_batch,y_:train_label_batch,keep_prob:0.5})
        
        #validation_loss=loss.eval(feed_dict={x:train_data_batch,y_:train_label_batch,keep_prob:1.0})
        validation_loss=loss.eval(feed_dict={x:train_data_batch,y_:train_label_batch,keep_prob:1.0})
        
        if validation_loss < best_validation_loss: 
            best_validation_loss=validation_loss
            cureent_epoch=i
        elif (i-current_epoch)>=EARLY_STOP_PATIENCE:
            #print ('early stopping')
            break
    print(best_validation_loss,end='')
    print(' ',end='')
        
y_pred=[]
    
test_loss,test_acc,batch_num=0,0,0
test_data_num=test_data.shape[0]
    
for test_data_batch in get_test_batch(test_data,batch_size):
    y_batch=y.eval(feed_dict={x:test_data_batch,keep_prob:1.0})
    y_pred.extend(y_batch)
    
    #如果测试数据不是batch_size的整数倍
if (test_data_num % batch_size) != 0:
    len_tmp=len(y_pred)
    for j in range(len_tmp,test_data_num):
        test_per=test_data[j,:]
        test_per=test_per.reshape(1,w,h,c)
        y_batch=y.eval(feed_dict={x:test_per,keep_prob:1.0})
        y_pred.extend(y_batch)
        
print('predict test image done!')
    
output_file=open('submission4.csv','w')
    
IdLookupTable=open('IdLookupTable.csv')
IdLookupTable.readline()
    
#output_file.write('{0},{1},{2},{3}\n'.format("RowId","ImageId","FeatureName","Location"))
output_file.write('{0},{1}\n'.format("RowId","Location"))
    
for line in IdLookupTable:
    RowId,ImageId,FeatureName=line.rstrip().split(',')
    #print("RowId:{},ImageId:{},FeatureName:{}".format(RowId,ImageId,FeatureName))
    image_index=int(ImageId)-1
    feature_index=keypoint_index[FeatureName]
    feature_location=y_pred[image_index][feature_index]*48+48
    #output_file.write('{0},{1},{2},{3}\n'.format(RowId,ImageId,FeatureName,feature_location))
    output_file.write('{0},{1}\n'.format(RowId,feature_location))

output_file.close()
IdLookupTable.close()
    
    

train_data.shape==(2140, 96, 96, 1);train_data.min==0.000;train_data.max=1.000
train_label.shape==(2140, 30);train_label.min==-0.920;train_label.max=0.996
test_data.shape==(1783, 96, 96, 1);
loss_begin
begin_inference1
begin_session
0.0709779 0.0522845 0.0413359 0.035148 0.0315759 0.0311076 

In [ ]:
test_data_num

In [ ]:
test_data,test_label1=load2d_show(test=True)
#test_data_num = 1
for i in range(test_data_num):
    print("image_show")
    test_img=test_data[i,:]
    test_label=y_pred[i]
    print(test_img)
    img2 = np.asarray(test_img, dtype=np.uint8)
    k=0
    for j in range(15):
        img2[int(test_label[k+1]*48+48)][int(test_label[k]*48+48)]=255
        #test_img=test_img[int(test_label[k+1]*48+48)][int(test_label[k]*48+48)]=255
        k=k+2
    cv2.imshow("image_test",img2)
    cv2.imshow("image_test_uint8",np.asarray(img2, dtype=np.uint8))
    cv2.imwrite("img.bmp",img2)
    cv2.waitKey(0)